# ***Priv Esc Runbook***

## Setup

In [ ]:
# will need to house these locally if they are not already in there, which I am sure some of them are.
# install packages
print("Installing Necesssary Packages, please wait.")
import sys
!{sys.executable} -m pip install kqlmagic --no-cache-dir --upgrade
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install datetime
!{sys.executable} -m pip install msticpy
!{sys.executable} -m pip install msticpy[azure,ml,kql,keyvault]
print("Package installation complete.")

In [ ]:
#import packages
print("Inporting Necesssary Packages, please wait.")
import Kqlmagic
import pandas as pd
from datetime import datetime
import msticpy
from msticpy.nbtools import process_tree as ptree
from msticpy.nbtools.timeline import display_timeline
print("Package import complete.")
%reload_ext Kqlmagic

## Variable Definitions

In [ ]:
#Adding IOCs as Global Variables to be used across functions
compromised_accounts = ('User1','User2')
compromised_machines = ('Computer1', 'Computer2')
start_time = '2022-03-08'
end_time = '2022-03-09'
privesc_tools = ("winPEAs", "adPEAs", "mimikatz", "powerup", "sherlock", "seatbelt", "jaws-enum", "wes.py", "privesccheck")
tool_used = ()
attribution = ()

## Adding Cluster Connections

In [ ]:
#add all necessary kusto connections
# will we need the full URL of the cluster? 
# i saw there was some pop-up auth, need ot make sure that will work for us too
%kql kusto://code().cluster('yourcluster').database('WEC')

## Queries

#### Initial WEC Query

In [ ]:
#queries
%%kql WEC@yourcluster
let usernames_list = compromised_accounts;
let start_time_string = start_time;
let end_time_string = end_time;
let start = todatetime(start_time_string);
let end = todatetime(end_time_string);
SecurityLog
| where ingestion_time() between(start .. end)
| where EventId == 4688
| where EventData.TargetUserName has_any (usernames_list)
| extend SubjectUserName = tostring(parse_json(EventData).SubjectUserName),
        TargetUserName = tostring(parse_json(EventData).TargetUserName),
        CommandLine = tostring(parse_json(EventData).CommandLine),
        ParentProcessName = tostring(parse_json(EventData).ParentProcessName),
        NewProcessName = tostring(parse_json(EventData).NewProcessName)
| project TimeCreated, Computer, SubjectUserName, TargetUserName, ParentProcessName, NewProcessName, CommandLine
| sort by TimeCreated desc
| limit 1000


#### WEC Results to DF Variable

In [ ]:
InitialWECLogs = _kql_raw_result_.to_dataframe()

#### Token Elevation Query

In [ ]:
# the %1937 tokenelevation type indicates a user running with admin or equivalent permissions, 
# similar to executing with runas
%%kql WEC@yourcluster
let start_time_string = start_time;
let end_time_string = end_time;
let start = todatetime(start_time_string);
let end = todatetime(end_time_string);
SecurityLog
| where ingestion_time() between(start..end)
| where EventId == 4688
| where Computer contains "JMP"
| where EventData.TokenElevation.Type contains "1937"
| sort by TimeCreated desc
| limit 1000

#### Token Elevation Results to DF Variable

In [ ]:
Token_Elevation = _kql_raw_result_.to_dataframe()

#### RunDLL Query

In [ ]:
# suspicious rundll32, this is a technique of dll hijacking
%%kql WEC@yourcluster
let start_time_string = start_time;
let end_time_string = end_time;
let start = todatetime(start_time_string);
let end = todatetime(end_time_string);
SecurityLog
| where ingestion_time() between(start_time..end_time)
| where EventId == 4688
| where EventData.CommandLine contains "rundll32"
| where EventData.NewProcessName !contains "system32"
| sort by TimeCreated desc
| limit 1000

#### RunDLL Results to DF Variable

In [ ]:
rundll_results = _kql_raw_result_.to_dataframe()

#### Pass the Hash Query

In [ ]:
# querying for possible pass the hash techniques - still working on this

%%kql WEC@yourcluster
let start_time_string = start_time;
let end_time_string = end_time;
let start = todatetime(start_time_string);
let end = todatetime(end_time_string);
SecurityLog
| where TimeCreated between(start..end)
| where EventId == 4624 and EventData.LogonType == 9
| where isnotempty(EventData.TargetOutBoundUserName) and EventData.TargetOutBoundUserName != "-"
| project TimeCreated, Computer, EventData.TargetOutBoundUserName, EventData.LogonType, EventData.IpAddress EventData.LogonGuid 
| join (SecurityLog
        | where TimeCreated between(start..end)
        | where EventId == 4648
        | where EventData.TargetServerName != "-" and EventData.TargetServerName != "localhost"
        | project TimeCreated, EventData.TargetServerName, Computer, EventData.TargetUserName, EventData.TargetInfo,
        EventData.LogonGiud
        ) on EventData.LogonGuid
| sort by TimeCreated desc

#### Pass the Hash Results to DF Variable

In [ ]:
pth_results = _kql_raw_result_.to_dataframe()

#### User Login Summary Query

In [ ]:
#user login summary
%%kql WEC@yourcluster
let usernames_list = compromised_accounts;
let start_time_string = start_time;
let end_time_string = end_time;
let start = todatetime(start_time_string);
let end = todatetime(end_time_string);
SecurityLog
| where ingestion_time() between(start..end)
| where EventId == 4624
| where Computer has_any (usernames_list)
| extend TargetUserName = tostring(EventData.TargetUserName),
    TargetUserSid = EventData.TargetUserSid,
    IpAddress = tostring(EventData.IpAddress),
    LogonType = tostring(EventData.LogonType)
| where TargetUserSid !~ 'S-1-5-18'
| summarize Count=count() by TargetUserName, Computer, IpAddress, LogonType
| sort by Count desc

#### User Login Results to DF Variable

In [ ]:
user_login_results = _kql_raw_result_.to_dataframe()

#### Computer Login Summary

In [ ]:
#users logged into particular machines 
%%kql WEC@yourcluster
let machine_list = compromised_machines;
let start_time_string = start_time;
let end_time_string = end_time;
let start = todatetime(start_time_string);
let end = todatetime(end_time_string);
SecurityLog
| where ingestion_time() between(start..end)
| where EventId == 4624
| where Computer has_any (machine_list)
| extend TargetUserName = tostring(EventData.TargetUserName),
    TargetUserSid = EventData.TargetUserSid,
    IpAddress = tostring(EventData.IpAddress),
    LogonType = tostring(EventData.LogonType)
| where TargetUserSid !~ 'S-1-5-18'
| summarize Count=count() by TargetUserName, Computer, IpAddress, LogonType
| sort by Count desc

#### Computer Login Results to DF Variable

In [ ]:
computer_login_results = _kql_raw_result_.to_dataframe()

#### Computer Processes

In [ ]:
# keep this timeframe short - the idea is to get a snapshot of the processes for the processtree
%%kql WEC@yourcluster
let machine_list = compromised_machines;
let start_time_string = start_time;
let end_time_string = end_time;
let start = todatetime(start_time_string);
let end = todatetime(end_time_string);
SecurityLog
| where ingestion_time() between(start..end)
| where EventId == 4688
| where Computer contains "computername"
| extend SubjectUserName = tostring(parse_json(EventData).SubjectUserName),
        TargetUserName = tostring(parse_json(EventData).TargetUserName),
        CommandLine = tostring(parse_json(EventData).CommandLine),
        ParentProcessName = tostring(parse_json(EventData).ParentProcessName),
        NewProcessName = tostring(parse_json(EventData).NewProcessName)
| project TimeCreated, Computer, SubjectUserName, TargetUserName, ParentProcessName, NewProcessName, CommandLine
| sort by TimeCreated desc
| limit 1000

#### Computer Processes Results to DF Variable

In [ ]:
computer_processes = _kql_raw_result_.to_dataframe()

## Visualizations

In [ ]:
#visualizations
#user processes
%%kql WEC@yourcluster
let username = "User1";
let start_time_string = start_time;
let end_time_string = end_time;
let start = todatetime(start_time_string);
let end = todatetime(end_time_string);
SecurityLog
| where ingestion_time() between(start..end)
| where EventId == 4688
| where EventData.SubjectUserName contains username or EventData.TargetUserName contains username
| summarize processes = count() by tostring(EventData.NewProcessName), bin(TimeCreated, 12h)
| render columnchart title= 'User Processes'

In [ ]:
#user logins
%%kql WEC@yourcluster
let username = "User1";
let start_time_string = start_time;
let end_time_string = end_time;
let start = todatetime(start_time_string);
let end = todatetime(end_time_string);    
SecurityLog
| where ingestion_time() between(start..end)
| where EventId == 4624
| where EventData.SubjectUserName contains username or EventData.TargetUserName contains username
| summarize logins = count() by tostring(Computer), bin(TimeCreated, 12h)
| render timechart title= 'User Logins'
        

In [ ]:
#machine logins 
%%kql WEC@yourcluster
let machine_list = compromised_machines;
let start_time_string = start_time;
let end_time_string = end_time;
let start = todatetime(start_time_string);
let end = todatetime(end_time_string);
SecurityLog
| where ingestion_time() between(start..end)
| where EventId == 4624
| where Computer has_any (machine_list)
| extend TargetUserName = tostring(EventData.TargetUserName),
    TargetUserSid = EventData.TargetUserSid,
    IpAddress = tostring(EventData.IpAddress),
    LogonType = tostring(EventData.LogonType)
| where TargetUserSid !~ 'S-1-5-18'
| summarize Count=count() by TargetUserName, Computer, IpAddress, LogonType
| render barchart

## Msticpy Functions 

In [ ]:
# create a process tree on a remote host
# will try to auto pull from data - if not in proper format it will use tree builder which you can also do manually
# will be using 4688 events - probably best to do this only over an hour increment unless you are narrowing by username
# the build tree extract parent/child relationships
# 3 examples for the same function

# computer_processes.mp_process_tree.plot()
ptree.plot_process_tree(computer_processes)

#if that doesnt work try this build function
# ptree.built_process_tree(computer_processes)

# can also do this for a user instead of a computer but it wont work if they escalate to system

In [ ]:
#create a simple process timeline - first it seems like data has to be put into a csv, then read from the CSV
#at least one column needs to be something like timecreated

# loading data source
host_processes(
   data=InitialWECLogs
   parse_dates=["TimeCreated"],
   infer_datetime_format=True,
   index_col=0
);
display_timeline(host_processes)

In [ ]:
# grouping by TargetUserName
display_timeline(
    host_processes,
    group_by="TargetUserName"
    source_columns=["ParentProcessName", "NewProcessName"],
    legend="inline");

## Data Analytics

In [ ]:
#all unique commandline arguments
unique_cmd = pd.unique(InitialWECLogs.CommandLine)
print(unique_cmd)

In [ ]:
#find any powershell encoded commands
encoded_filter = InitialWECLogs['CommandLine'].str.find("encodedcommand")
encoded_filter

In [ ]:
#number of processes spawned by each parent
#this is something we could baseline as well - notate the values over the course of a normal week for example and
proc_relationship = InitialWECLogs.groupby(['ParentProcessName']).size()
proc_relationship